##**objective**

because academic literature review suggested that multi-mode transport (MMT) detection is difficult to predict, i decided moving towards predicting single mode transport (SMT) instead. 

this notebook creates 2 files for SMT modeling


*   mtl_trajet
*   mtl_pts



####**dependencies + packages**

In [1]:
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirement
!pip install descartes 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
python3-gdal is already the newest version (2.2.3+dfsg-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-resources
Suggested packages:
  python3-setuptools
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-

In [0]:
from shapely.geometry import MultiLineString

import geopandas as gpd
import os
import pandas as pd

####**mtl_trajet**

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
# change directory
drive.mount('/content/drive', force_remount= True)
os.chdir('/content/drive/My Drive/MRP/mtl trajet data')

Mounted at /content/drive


In [0]:
# set path for geojson files
mtl_trajet_json = 'trajets_mtl_trajet_2017-1.geojson'
mtl_pts_json = 'points_mtl_trajet_2017-1.geojson'

In [8]:
# import file
mtl_trajet = gpd.read_file(mtl_trajet_json)
print(mtl_trajet.head())

   id_trip  ...                                           geometry
0     1547  ...  MULTILINESTRING ((-73.65117 45.54450, -73.6512...
1   308312  ...  MULTILINESTRING ((-73.65099 45.54450, -73.6510...
2   384772  ...  MULTILINESTRING ((-73.83287 45.63607, -73.8317...
3   150744  ...  MULTILINESTRING ((-73.64638 45.54629, -73.6460...
4   199011  ...  MULTILINESTRING ((-73.74550 45.55839, -73.7454...

[5 rows x 6 columns]


In [9]:
# columns within file
mtl_trajet.columns

Index(['id_trip', 'starttime', 'endtime', 'mode', 'purpose', 'geometry'], dtype='object')

In [10]:
mtl_trajet[['id_trip', 'starttime', 'endtime', 'mode', 'purpose']].head()

,id_trip,starttime,endtime,mode,purpose
0,1547,2017-09-18T04:16:58,2017-09-18T04:26:26,None,None
1,308312,2017-09-18T06:17:46,2017-09-18T06:58:49,None,None
2,384772,2017-09-18T09:30:24,2017-09-18T10:17:28,None,None
3,150744,2017-09-18T10:02:50,2017-09-18T10:17:12,Voiture / Moto,Reconduire / aller chercher une personne
4,199011,2017-09-18T10:18:40,2017-09-18T10:27:44,None,None


In [12]:
# determine the min/max dates to use for weather summary data extract

print ('min date : {}'.format(mtl_trajet['starttime'].min()))
print ('max date : {}'.format(mtl_trajet['starttime'].max()))

min date : 2017-09-18T04:16:58
max date : 2017-10-18T03:50:08


In [14]:
# query all modes of transportation within dataset
# values with multiple modes indicate MMT
print ('no. of modes of transportation : {}\n'.format(len(mtl_trajet['mode'].unique())))
mtl_trajet['mode'].unique()

no. of modes of transportation : 70



array([None, 'Voiture / Moto', 'À pied, Transport collectif',
       'Transport collectif', 'À pied', 'Vélo',
       'À pied, Transport collectif, Vélo',
       'Transport collectif, Voiture / Moto', 'À pied, Vélo',
       'À pied, Voiture / Moto, Vélo', 'Autopartage', 'Autre',
       'Transport collectif, Vélo',
       'À pied, Transport collectif, Voiture / Moto',
       'Transport collectif, Voiture / Moto, Vélo',
       'À pied, Voiture / Moto', 'Voiture / Moto, Vélo', 'Taxi', 'ND',
       'Taxi, Vélo', 'À pied, Autopartage, Vélo',
       'Autopartage, Transport collectif, Voiture / Moto',
       'Taxi, Voiture / Moto', 'À pied, Autopartage, Voiture / Moto',
       'Autopartage, Autre', 'À pied, Taxi, Vélo', 'Autopartage, Taxi',
       'Autopartage, Voiture / Moto',
       'Autopartage, Autre, Voiture / Moto',
       'Autopartage, Transport collectif',
       'À pied, Transport collectif, Voiture / Moto, Vélo',
       'Autre, Transport collectif', 'À pied, Autre, Transport collecti

In [15]:
# query the frequency of each transportation mode
mtl_trajet['mode'].value_counts()

Voiture / Moto                                                                 24744
Vélo                                                                           13595
Transport collectif                                                            12935
À pied                                                                         11062
À pied, Transport collectif                                                     6234
                                                                               ...  
Taxi, Transport collectif, Vélo                                                    1
Autopartage, Autre, Voiture / Moto                                                 1
À pied, Autopartage, Autre, Taxi, Transport collectif, Voiture / Moto, Vélo        1
À pied, Taxi, Transport collectif, Voiture / Moto, Vélo                            1
À pied, Autre, Voiture / Moto                                                      1
Name: mode, Length: 69, dtype: int64

In [0]:
# calculate no. of modes of transportation
mtl_trajet['num_modes_trans'] = mtl_trajet['mode'].str.count(',') + 1

In [0]:
mtl_trajet[['id_trip', 'starttime', 'endtime', 'mode', 'purpose', 'num_modes_trans']].head()

,id_trip,starttime,endtime,mode,purpose,num_modes_trans
0,1547,2017-09-18T04:16:58,2017-09-18T04:26:26,None,None,NaN
1,308312,2017-09-18T06:17:46,2017-09-18T06:58:49,None,None,NaN
2,384772,2017-09-18T09:30:24,2017-09-18T10:17:28,None,None,NaN
3,150744,2017-09-18T10:02:50,2017-09-18T10:17:12,Voiture / Moto,Reconduire / aller chercher une personne,1.0
4,199011,2017-09-18T10:18:40,2017-09-18T10:27:44,None,None,NaN


In [0]:
mtl_trajet['num_modes_trans'] = mtl_trajet['num_modes_trans'].fillna(0)

In [0]:
# check
mtl_trajet[['mode', 'num_modes_trans']][mtl_trajet['num_modes_trans'] == 2].head()

,mode,num_modes_trans
15,"À pied, Transport collectif",2.0
51,"Transport collectif, Voiture / Moto",2.0
55,"À pied, Vélo",2.0
74,"À pied, Vélo",2.0
100,"À pied, Vélo",2.0


In [0]:
# no. of single mode transport trips (SMT)
mtl_trajet['num_modes_trans'].value_counts()

0.0    111067
1.0     63962
2.0      9296
3.0       931
4.0        27
5.0         1
7.0         1
Name: num_modes_trans, dtype: int64

In [0]:
print ('no. of trips (collected) : {}'.format(len(mtl_trajet)))

no. of trips (collected) : 185285


In [0]:
# query SMT
mtl_trajet_SMT = mtl_trajet[mtl_trajet['num_modes_trans'] == 1]
mtl_trajet_SMT.loc[:, mtl_trajet_SMT.columns != 'geometry'].head()

,id_trip,starttime,endtime,mode,purpose,num_modes_trans
3,150744,2017-09-18T10:02:50,2017-09-18T10:17:12,Voiture / Moto,Reconduire / aller chercher une personne,1.0
13,173651,2017-09-18T11:14:10,2017-09-18T12:01:46,Voiture / Moto,Travail / Rendez-vous d'affaires,1.0
16,149086,2017-09-18T11:20:42,2017-09-18T12:10:14,Transport collectif,Travail / Rendez-vous d'affaires,1.0
19,358412,2017-09-18T11:27:38,2017-09-18T11:40:26,À pied,Travail / Rendez-vous d'affaires,1.0
20,350121,2017-09-18T11:31:56,2017-09-18T12:11:26,Voiture / Moto,Travail / Rendez-vous d'affaires,1.0


In [0]:
print ('no. of trips (SMT) : {}'.format(len(mtl_trajet_SMT)))

no. of trips (SMT) : 63962


####**mtl_pts**

In [18]:
# import file
mtl_pts = gpd.read_file(mtl_pts_json)
print(mtl_pts.head())

    latitude  longitude  ...  id_trip                    geometry
0  45.545447 -73.653716  ...     1547  POINT (-73.65372 45.54545)
1  45.545756 -73.653425  ...     1547  POINT (-73.65342 45.54576)
2  45.545652 -73.653715  ...     1547  POINT (-73.65372 45.54565)
3  45.545570 -73.654042  ...     1547  POINT (-73.65404 45.54557)
4  45.545406 -73.654266  ...     1547  POINT (-73.65427 45.54541)

[5 rows x 9 columns]


In [20]:
# no. of unique points
print ('total no. of GPS coordinates : {}'.format(len(mtl_pts['geometry'])))
print ('no. of unique GPS coordinates : {}'.format(len(mtl_pts['geometry'].unique())))

total no. of GPS coordinates : 13015782
no. of unique GPS coordinates : 12385134


In [0]:
# merge file with mtl_trajet_SMT in order to obtain all SMT trips within mtl_pts
mtl_pts = pd.merge(mtl_pts, mtl_trajet_SMT['id_trip'], how = 'inner', on = 'id_trip')

In [0]:
# get columns
mtl_pts.columns

Index(['latitude', 'longitude', 'speed', 'altitude', 'h_accuracy',
       'v_accuracy', 'timestamp', 'id_trip', 'geometry'],
      dtype='object')

In [0]:
mtl_pts[['latitude', 'longitude', 'speed', 'altitude', 'h_accuracy', 'v_accuracy', 'timestamp', 'id_trip', 'geometry']].head()

,latitude,longitude,speed,altitude,h_accuracy,v_accuracy,timestamp,id_trip,geometry
0,45.546167,-73.646082,8.31,37.781403,10.0,4.0,2017-09-18T10:03:52,150744,POINT (-73.64608 45.54617)
1,45.545952,-73.645344,13.60,38.696411,10.0,4.0,2017-09-18T10:03:57,150744,POINT (-73.64534 45.54595)
2,45.545662,-73.644330,14.79,39.560181,10.0,4.0,2017-09-18T10:04:03,150744,POINT (-73.64433 45.54566)
3,45.545451,-73.643589,11.87,39.500397,10.0,4.0,2017-09-18T10:04:08,150744,POINT (-73.64359 45.54545)
4,45.545226,-73.642798,11.63,40.618835,10.0,4.0,2017-09-18T10:04:15,150744,POINT (-73.64280 45.54523)


In [0]:
# export file
mtl_pts.to_file('mtl_pts_SMT_2017.geojson', driver = 'GeoJSON')

In [0]:
print ('no. of trips (SMT) : {}'.format(len(mtl_pts['id_trip'].unique())))

no. of trips (SMT) : 62771


In [0]:
# because no. of SMT trips did not match up, we'll need to query mtl_trajet_SMT with the unique id_trips within the mtl_pts file
# query all unique trip ids
id_trips = mtl_pts['id_trip'].unique()

In [0]:
# match all id_trips
mtl_trajet_SMT = mtl_trajet_SMT[mtl_trajet_SMT['id_trip'].isin(id_trips)]

In [0]:
print ('no. of trips (SMT) : {}'.format(len(mtl_trajet_SMT)))

no. of trips (SMT) : 62771


In [0]:
# export file
mtl_trajet_SMT.to_file('mtl_trajet_SMT_2017.geojson', driver = 'GeoJSON')